In [1]:
# Importing Libraries

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
from keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [3]:
from google.colab import files
uploaded = files.upload()

Saving bbc.csv to bbc.csv


In [4]:
# Reading dataset into dataframe
data = pd.read_csv("bbc.csv")
data.head()
data.rename(columns={'Unnamed':'index'},inplace=True)
data.head()
data1 = data.copy()

In [5]:
# Creating new Feature Class ID for Dataframe
data['Class_id'] = data['Class'].factorize()[0]
data.groupby('Class').Class_id.count()

Class
business         510
entertainment    386
politics         417
sport            198
tech             401
Name: Class_id, dtype: int64

In [6]:
# Converting data into vector to make it suitable for input to neuron
np.random.seed(1)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(data.Article).toarray()
features
label = data.Class_id
label

0       0
1       0
2       0
3       0
4       0
       ..
1907    4
1908    4
1909    4
1910    4
1911    4
Name: Class_id, Length: 1912, dtype: int64

# First Implementation With 2 Hidden layer and 5 epochs

In [7]:
def tanh(x):
    return np.tanh(x)

def d_tanh(x):
    return 1 - np.square(np.tanh(x))

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def d_sigmoid(x):
    return (1 - sigmoid(x)) * sigmoid(x)
'''def sigmoid(z):
    g = np.frompyfunc(lambda x: 1 / (1 + np.exp(-x)), 1, 1)
    return g(z).astype(z.dtype)


def sigmoidGradient(z):
    return sigmoid(z) * (1 - sigmoid(z))'''

'def sigmoid(z):\n    g = np.frompyfunc(lambda x: 1 / (1 + np.exp(-x)), 1, 1)\n    return g(z).astype(z.dtype)\n\n\ndef sigmoidGradient(z):\n    return sigmoid(z) * (1 - sigmoid(z))'

In [9]:
def initializeWeights(L_in, L_out):
  epsilon_init = 0.12
  W = np.random.rand(L_out, 1 + L_in) * 2 * \
     epsilon_init - epsilon_init
  return W

In [18]:
X = features # np.matrix(df.to_numpy())
y = label.to_numpy().transpose() # np.matrix(Y.to_numpy()).transpose()
m,n = X.shape
input_layer_size = 12860                     # Dimension of features
hidden_layer_size = 50*2    # of units in hidden layer
output_layer_size = 5     # number of labels

In [8]:
# Implementing Feed Forward Neural Network

W1 = np.random.randn(12860,50)
b1 = np.zeros((1,50))

W2 = np.random.randn(50,50)
b2 = np.zeros((1,50))
W3 = np.random.randn(50,5)
b3 = np.zeros((1,5))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, label, data.index, test_size=0.20, random_state=0)
Z1 = X_train.dot(W1) + b1
A1 = tanh(Z1)

Z2 = A1.dot(W2) + b2
A2 = tanh(Z2)

Z3 = A2.dot(W3) + b3
A3 = sigmoid(Z3)

print(A3)

[[2.74601994e-02 9.97634978e-01 4.60793500e-01 3.29728540e-03
  1.31484832e-02]
 [3.21521851e-03 9.99778710e-01 2.26621459e-04 9.95596081e-05
  2.87610261e-02]
 [4.39233413e-04 9.97996806e-01 9.99549561e-01 2.97482632e-02
  5.70736023e-03]
 ...
 [3.41701356e-01 9.99742244e-01 7.33849728e-06 9.71375437e-01
  6.35136762e-01]
 [9.97496391e-01 9.81949658e-01 1.65828954e-03 9.93875798e-01
  9.93732019e-01]
 [9.98901423e-01 2.29302267e-01 8.26074302e-02 1.06642667e-01
  9.99469923e-01]]


In [9]:
# Implementing general Model of Neural Network
id_to_category = {0: 'business', 1: 'tech', 2: 'politics', 3: 'sport', 4: 'entertainment'}
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, label, data.index, test_size=0.20, random_state=0)
  
model1 = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(50, 50), random_state=6,max_iter = 200)
model1.fit(X_train,y_train)
y_pred = model1.predict(X_test)
print(accuracy_score(y_test, y_pred))
p=0
TP=0
FP=0
FN=0
TN=0
for j in range(0,5):
  p=0
  for i in indices_test:
    if(y_test[i]==j):
      if(y_test[i] == y_pred[p]):
       TP+=1
      else:
        FN+=1
    else:
      if(y_test[i] == y_pred[p]):
        TN+=1
      else:
        if(y_pred[p]==j):
          FP+=1
    p+=1

  recall =  TP/(TP+FN)
  precision = TP/(TP + FP)
  print("Precsion And Recall value for " + str(id_to_category[j]+" Class"))
  print("Precision: "+str(precision))
  print("Recall: " + str(recall))





0.9686684073107049
Precsion And Recall value for business Class
Precision: 0.9649122807017544
Recall: 0.9734513274336283
Precsion And Recall value for tech Class
Precision: 0.9685863874345549
Recall: 0.9788359788359788
Precsion And Recall value for politics Class
Precision: 0.9705882352941176
Recall: 0.9705882352941176
Precsion And Recall value for sport Class
Precision: 0.9714285714285714
Recall: 0.9683544303797469
Precsion And Recall value for entertainment Class
Precision: 0.9686684073107049
Recall: 0.9686684073107049


In [10]:
# KFold Cross Validation
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3,shuffle=True,random_state=1)
fdata = list(features)
ldata = list(label)

In [39]:
Y_test = np.array(Y_test).reshape(-1,1)

Class
business         510
entertainment    386
politics         417
sport            198
tech             401

Model With 2 Hidden Layer and Tanh as Hidden Layer Activation and 3 Fold Validation

In [12]:
i=1
Accuracy = 0
class_accur_bus=0
class_accur_entr=0
class_accur_pol=0
class_accur_spo=0
class_accur_tec = 0
from sklearn.neural_network import MLPClassifier
for train,test in kfold.split(features):
  #print(type(train))
  X_train,X_test,Y_train,Y_test = features[train],features[test],label[train],label[test]
  #Y_test = np.array(Y_test).reshape(-1,1)
  
  model2 = MLPClassifier(solver='lbfgs',activation='tanh',alpha=1e-5,hidden_layer_sizes=(50, 50), random_state=6)
  model2.fit(X_train,Y_train)
  Y_pred = model2.predict(X_test)
  Accuracy += (accuracy_score(Y_test, Y_pred))
  p=0
  for j in range(0,5):
    t=0
    pr=0
    p=0
    TP=0
    for i in test:
      if(Y_test[i]==j):
        #print("Y_test: "+str(Y_test[i]))
        #print("Y_pred: "+str(Y_pred[p]))
        t+=1
        if(Y_pred[p]==Y_test[i]):
          TP+=1
      p+=1
    #print(t/TP)
    if(j==0):
      class_accur_bus += (TP/t)
    if(j==1):
      class_accur_entr += (TP/t)
    if(j==2):
      class_accur_pol += (TP/t)
    if(j==3):
      class_accur_spo += (TP/t)
    if(j==4):
      class_accur_tec += (TP/t)
print('Performance Measure for Model with Tanh as hidden layer and 50 neurons per Hidden Layer and lbfgs optimiser')
print("Overall Accuracy for this Model after 3 Fold Validation: "+str(Accuracy/3))
class_accur_bus=class_accur_bus/3
print("Overall Accuracy value for Business Class: "+str(class_accur_bus))
class_accur_entr = class_accur_entr/3
print("Overall Accuracy value for Entertainment Class: "+str(class_accur_entr))
class_accur_pol = class_accur_pol/3
print("Overall Accuracy value for Politics Class: "+str(class_accur_pol))
class_accur_spo = class_accur_spo/3
print("Overall Accuracy value for Sports Class: "+str(class_accur_spo))
class_accur_tec = class_accur_tec/3
print("Overall Accuracy value for Technical Class: "+str(class_accur_tec))

    







Performance Measure for Model with Tanh as hidden layer and 50 neurons per Hidden Layer
Overall Accuracy for this Model after 3 Fold Validation: 0.9754162094063572
Overall Accuracy value for Business Class: 0.9630742442690128
Overall Accuracy value for Entertainment Class: 0.978973500424193
Overall Accuracy value for Politics Class: 0.9711872089054191
Overall Accuracy value for Sports Class: 0.9951690821256038
Overall Accuracy value for Technical Class: 0.9828101493660005


Model With Adam Optimizer

In [14]:
i=1
Accuracy = 0
class_accur_bus=0
class_accur_entr=0
class_accur_pol=0
class_accur_spo=0
class_accur_tec = 0
from sklearn.neural_network import MLPClassifier
for train,test in kfold.split(features):
  #print(type(train))
  X_train,X_test,Y_train,Y_test = features[train],features[test],label[train],label[test]
  #Y_test = np.array(Y_test).reshape(-1,1)
  
  model3 = MLPClassifier(solver='adam',activation='tanh',alpha=1e-5,hidden_layer_sizes=(50, 50), random_state=6)
  model3.fit(X_train,Y_train)
  Y_pred = model3.predict(X_test)
  Accuracy += (accuracy_score(Y_test, Y_pred))
  p=0
  for j in range(0,5):
    t=0
    pr=0
    p=0
    TP=0
    for i in test:
      if(Y_test[i]==j):
        #print("Y_test: "+str(Y_test[i]))
        #print("Y_pred: "+str(Y_pred[p]))
        t+=1
        if(Y_pred[p]==Y_test[i]):
          TP+=1
      p+=1
    #print(t/TP)
    if(j==0):
      class_accur_bus += (TP/t)
    if(j==1):
      class_accur_entr += (TP/t)
    if(j==2):
      class_accur_pol += (TP/t)
    if(j==3):
      class_accur_spo += (TP/t)
    if(j==4):
      class_accur_tec += (TP/t)
print('Performance Measure for Model with Tanh as hidden layer and 50 neurons per Hidden Layer and ADAM Optimiser')
print("Overall Accuracy for this Model after 3 Fold Validation: "+str(Accuracy/3))
class_accur_bus=class_accur_bus/3
print("Overall Accuracy value for Business Class: "+str(class_accur_bus))
class_accur_entr = class_accur_entr/3
print("Overall Accuracy value for Entertainment Class: "+str(class_accur_entr))
class_accur_pol = class_accur_pol/3
print("Overall Accuracy value for Politics Class: "+str(class_accur_pol))
class_accur_spo = class_accur_spo/3
print("Overall Accuracy value for Sports Class: "+str(class_accur_spo))
class_accur_tec = class_accur_tec/3
print("Overall Accuracy value for Technical Class: "+str(class_accur_tec))

    







Performance Measure for Model with Tanh as hidden layer and 50 neurons per Hidden Layer and ADAM Optimiser
Overall Accuracy for this Model after 3 Fold Validation: 0.9790751120800382
Overall Accuracy value for Business Class: 0.9730661054597664
Overall Accuracy value for Entertainment Class: 0.9814040955402158
Overall Accuracy value for Politics Class: 0.980887719173896
Overall Accuracy value for Sports Class: 0.9951690821256038
Overall Accuracy value for Technical Class: 0.9754744922630562


In [15]:
i=1
Accuracy = 0
class_accur_bus=0
class_accur_entr=0
class_accur_pol=0
class_accur_spo=0
class_accur_tec = 0
from sklearn.neural_network import MLPClassifier
for train,test in kfold.split(features):
  #print(type(train))
  X_train,X_test,Y_train,Y_test = features[train],features[test],label[train],label[test]
  #Y_test = np.array(Y_test).reshape(-1,1)
  
  model4 = MLPClassifier(solver='sgd',activation='tanh',alpha=1e-5,hidden_layer_sizes=(50, 50), random_state=6)
  model4.fit(X_train,Y_train)
  Y_pred = model4.predict(X_test)
  Accuracy += (accuracy_score(Y_test, Y_pred))
  p=0
  for j in range(0,5):
    t=0
    pr=0
    p=0
    TP=0
    for i in test:
      if(Y_test[i]==j):
        #print("Y_test: "+str(Y_test[i]))
        #print("Y_pred: "+str(Y_pred[p]))
        t+=1
        if(Y_pred[p]==Y_test[i]):
          TP+=1
      p+=1
    #print(t/TP)
    if(j==0):
      class_accur_bus += (TP/t)
    if(j==1):
      class_accur_entr += (TP/t)
    if(j==2):
      class_accur_pol += (TP/t)
    if(j==3):
      class_accur_spo += (TP/t)
    if(j==4):
      class_accur_tec += (TP/t)
print('Performance Measure for Model with Tanh as hidden layer and 50 neurons per Hidden Layer and SGD Optimiser')
print("Overall Accuracy for this Model after 3 Fold Validation: "+str(Accuracy/3))
class_accur_bus=class_accur_bus/3
print("Overall Accuracy value for Business Class: "+str(class_accur_bus))
class_accur_entr = class_accur_entr/3
print("Overall Accuracy value for Entertainment Class: "+str(class_accur_entr))
class_accur_pol = class_accur_pol/3
print("Overall Accuracy value for Politics Class: "+str(class_accur_pol))
class_accur_spo = class_accur_spo/3
print("Overall Accuracy value for Sports Class: "+str(class_accur_spo))
class_accur_tec = class_accur_tec/3
print("Overall Accuracy value for Technical Class: "+str(class_accur_tec))

    







/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Performance Measure for Model with Tanh as hidden layer and 50 neurons per Hidden Layer and SGD Optimiser
Overall Accuracy for this Model after 3 Fold Validation: 0.7615258304913478
Overall Accuracy value for Business Class: 0.9920406591344495
Overall Accuracy value for Entertainment Class: 0.7567237624256485
Overall Accuracy value for Politics Class: 0.7520991762462469
Overall Accuracy value for Sports Class: 0.0
Overall Accuracy value for Technical Class: 0.8651609176875135


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Model with RELU as Activation Function and ADAM Optimiser

In [17]:
i=1
Accuracy = 0
class_accur_bus=0
class_accur_entr=0
class_accur_pol=0
class_accur_spo=0
class_accur_tec = 0
from sklearn.neural_network import MLPClassifier
for train,test in kfold.split(features):
  #print(type(train))
  X_train,X_test,Y_train,Y_test = features[train],features[test],label[train],label[test]
  #Y_test = np.array(Y_test).reshape(-1,1)
  
  model5 = MLPClassifier(solver='adam',activation='relu',alpha=1e-5,hidden_layer_sizes=(50, 50), random_state=6)
  model5.fit(X_train,Y_train)
  Y_pred = model5.predict(X_test)
  Accuracy += (accuracy_score(Y_test, Y_pred))
  p=0
  for j in range(0,5):
    t=0
    pr=0
    p=0
    TP=0
    for i in test:
      if(Y_test[i]==j):
        #print("Y_test: "+str(Y_test[i]))
        #print("Y_pred: "+str(Y_pred[p]))
        t+=1
        if(Y_pred[p]==Y_test[i]):
          TP+=1
      p+=1
    #print(t/TP)
    if(j==0):
      class_accur_bus += (TP/t)
    if(j==1):
      class_accur_entr += (TP/t)
    if(j==2):
      class_accur_pol += (TP/t)
    if(j==3):
      class_accur_spo += (TP/t)
    if(j==4):
      class_accur_tec += (TP/t)
print('Performance Measure for Model with Relu as hidden layer activation Function and 50 neurons per Hidden Layer and ADAM Optimiser')
print("Overall Accuracy for this Model after 3 Fold Validation: "+str(Accuracy/3))
class_accur_bus=class_accur_bus/3
print("Overall Accuracy value for Business Class: "+str(class_accur_bus))
class_accur_entr = class_accur_entr/3
print("Overall Accuracy value for Entertainment Class: "+str(class_accur_entr))
class_accur_pol = class_accur_pol/3
print("Overall Accuracy value for Politics Class: "+str(class_accur_pol))
class_accur_spo = class_accur_spo/3
print("Overall Accuracy value for Sports Class: "+str(class_accur_spo))
class_accur_tec = class_accur_tec/3
print("Overall Accuracy value for Technical Class: "+str(class_accur_tec))

    







Performance Measure for Model with Relu as hidden layer activation Function and 50 neurons per Hidden Layer and ADAM Optimiser
Overall Accuracy for this Model after 3 Fold Validation: 0.9785551066339244
Overall Accuracy value for Business Class: 0.9730410126162452
Overall Accuracy value for Entertainment Class: 0.9809835121911106
Overall Accuracy value for Politics Class: 0.980887719173896
Overall Accuracy value for Sports Class: 0.9951690821256038
Overall Accuracy value for Technical Class: 0.972551310982162
